# Detail Rocket Deep Learning Recommendation System

## Imports and installs

In [67]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
#%pip install pandas
# %pip install seaborn
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [68]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import operator
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
#test

## Load Data

Preprocessing was done in EDA notebook and stored in pickle files

In [69]:
basepath = '../datasets/preprocessed_datasets/retailrocket/'
category_tree = pd.read_pickle(basepath +'category_tree.pkl')
category_tree

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0
...,...,...
1664,49,1125.0
1665,1112,630.0
1666,1336,745.0
1667,689,207.0


In [70]:
events = pd.read_pickle(basepath +'events.pkl')
events

,timestamp,visitorid,event,itemid,transactionid,datetime
0,1433221332117,257597,view,355908,NaN,2015-06-02 07:02:12
1,1433224214164,992329,view,248676,NaN,2015-06-02 07:50:14
2,1433221999827,111016,view,318965,NaN,2015-06-02 07:13:19
3,1433221955914,483717,view,253185,NaN,2015-06-02 07:12:35
4,1433221337106,951259,view,367447,NaN,2015-06-02 07:02:17
...,...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN,2015-08-01 05:13:05
2756097,1438399813142,762376,view,115946,NaN,2015-08-01 05:30:13
2756098,1438397820527,1251746,view,78144,NaN,2015-08-01 04:57:00
2756099,1438398530703,1184451,view,283392,NaN,2015-08-01 05:08:50


In [71]:
item_properties = pd.read_pickle(basepath +'item_properties.pkl')
item_properties

,timestamp,itemid,property,value,datetime
0,1435460400000,460429,categoryid,1338,2015-06-28 05:00:00
1,1441508400000,206783,888,1116713 960601 n277.200,2015-09-06 05:00:00
2,1439089200000,395014,400,n552.000 639502 n720.000 424566,2015-08-09 05:00:00
3,1431226800000,59481,790,n15360.000,2015-05-10 05:00:00
4,1431831600000,156781,917,828513,2015-05-17 05:00:00
...,...,...,...,...,...
9275898,1433646000000,236931,929,n12.000,2015-06-07 05:00:00
9275899,1440903600000,455746,6,150169 639134,2015-08-30 05:00:00
9275900,1439694000000,347565,686,610834,2015-08-16 05:00:00
9275901,1433646000000,287231,867,769062,2015-06-07 05:00:00


In [72]:
visitor_actions = pd.read_pickle(basepath +'visitor_actions.pkl')
visitor_actions

,visitorid,products_viewed,num_views,num_add2cart,num_transaction
0,693516,1,1,2,0
1,829044,1,1,0,0
2,652699,1,1,0,0
3,1125936,1,1,0,0
4,1149227,1,1,0,0
...,...,...,...,...,...
1407575,998517,1,1,0,0
1407576,472345,1,1,0,0
1407577,1207677,1,1,0,0
1407578,255126,1,1,0,0


In [73]:
product_item_df = pd.read_pickle(basepath +'product_item.pkl')
product_item_df

,itemid,visitorid,view,addtocart,transaction
0,3,370720,1,0,0
1,3,639016,1,0,0
2,4,905555,1,0,0
3,4,1010132,1,0,0
4,4,1042455,1,0,0
...,...,...,...,...,...
2145174,466864,1388296,4,0,0
2145175,466867,241803,1,0,0
2145176,466867,450511,1,0,0
2145177,466867,1001859,1,0,0


In [74]:
len(product_item_df.visitorid.unique()), len(product_item_df.itemid.unique())

(1407580, 235061)

# Training Stuff

In [75]:
X = product_item_df[['view', 'addtocart']]
y = product_item_df['transaction']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4, test_size=0.3)


In [76]:
class Recommender(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim, attention_size):
        super(Recommender, self).__init__()
        self.user_embedding = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embedding = tf.keras.layers.Embedding(num_items, embedding_dim)
        self.attention_layer = tf.keras.layers.Attention(use_scale=True, input_dim=attention_size)
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        user_ids, item_ids = inputs[:, 0], inputs[:, 1]
        user_embeddings = self.user_embedding(user_ids)
        item_embeddings = self.item_embedding(item_ids)
        attention_weights = self.attention_layer([user_embeddings, item_embeddings])
        weighted_embeddings = attention_weights * item_embeddings
        output = self.output_layer(weighted_embeddings)
        return output

In [77]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [78]:
num_users = len(product_item_df.visitorid.unique())
num_items = len(product_item_df.itemid.unique())
num_epochs = 2

In [79]:
model = Recommender(num_users, num_items, 32, 64)
model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train,
    epochs=num_epochs,
    validation_data=y_train,
)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")